In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix, plot_confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

In [3]:
#Import Data
df = pd.read_csv('../Data/Sentiment CSV/MergedSenti.csv')
df.head()

,neg,neu,pos,compound,Post,Class
0,0.163,0.658,0.178,0.1406,attempted yesterday i had an exit yesterday an...,SuicidalThoughts
1,0.000,0.892,0.108,0.4404,my bf of years just tried to hang himself las...,SuicidalThoughts
2,0.141,0.665,0.194,0.8683,he gave up because of me we have shared many ...,SuicidalThoughts
3,0.045,0.873,0.081,0.2960,running out of options does anyone have any co...,SuicidalThoughts
4,0.299,0.588,0.113,-0.9927,my life is shit i feel like shit i hate mysel...,SuicidalThoughts


In [3]:
#Check the shape dataframe
df.shape

(51907, 6)

In [4]:
#Check the classes
df.Class.value_counts(normalize = True)

Exercise            0.281677
SuicidalThoughts    0.268076
Depression          0.264165
Happy               0.186083
Name: Class, dtype: float64

### Create with Model (Both Depression & Suicide Class)

In [9]:
#Reduce the dataset size
exercise_df = df[df['Class'] == 'Exercise'][:3000].reset_index(drop = True)
happy_df = df[df['Class'] == 'Happy'][:2000].reset_index(drop = True)
depression_df = df[df['Class'] == 'Depression'][:3000].reset_index(drop = True)
suicide_df = df[df['Class'] == 'SuicidalThoughts'][:2000].reset_index(drop = True)

In [10]:
comb_df = pd.concat([exercise_df, happy_df, depression_df, suicide_df]).reset_index(drop = True)
comb_df.shape

(10000, 6)

In [214]:
comb_df['Class'] = comb_df['Class'].apply(lambda x: 1 if x == 'SuicidalThoughts' or x == 'Depression'  else 0)

In [215]:
comb_df['Class'].value_counts(normalize = True)

0    0.5
1    0.5
Name: Class, dtype: float64

In [255]:
X = comb_df['Post'].to_list()
y = comb_df['Class'].to_list()

In [256]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [257]:
#Bert Layers
input_text = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(input_text)
outputs = bert_encoder(preprocessed_text)

#Create some hidden layers and dropot layers
h1 = tf.keras.layers.Dense(64, activation = 'relu', name = 'hidden_layer1')(outputs['pooled_output'])
d1 = tf.keras.layers.Dropout(0.5, name = 'Dropout1')(h1)
h2 = tf.keras.layers.Dense(64, activation = 'relu', name = 'hidden_layer2')(d1)
d2 = tf.keras.layers.Dropout(0.5, name = 'Dropout2')(h2)
h3 = tf.keras.layers.Dense(64, activation = 'relu', name = 'hidden_layer3')(d2)
d3 = tf.keras.layers.Dropout(0.5, name = 'Dropout3')(h3)
out_layer = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output_layer')(d3)

#Create the model
model_2 = tf.keras.Model(inputs = input_text, outputs = out_layer)

#Check the summary of model
model_2.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer_2 (KerasLayer)     {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [258]:
model_2.compile(loss = loss_func, optimizer = optimizer, metrics = [accuracy])

In [4]:
history_2 = model_2.fit(X_train, y_train,
                   batch_size = 64,
                    validation_data = (X_test, y_test),
                    epochs = 5)

In [ ]:
model.save('..')